Progetto: **Classificazione di testi descrittivi per destinazione d'uso, inerenti ai beni sottoposti ad aste giudiziarie italiane.**

Studente: **Alessandro Monolo** | *10439147*

Relatore: Marco Brambilla

Referente aziendale: Simone Redaelli

Master: Data Science & Artificial Intelligence

Università: Politecnico di Milano

<hr style="border:1px solid black">

# Studio delle variabili, prima fase di pulizia e sanatoria delle Null Values per le colonne numeriche

- **1.** Controllo le varie Colonne, una per una. Se trovo errori o dati ambigui, cerco come correggerli, ove possibile;
    - **1.1** - Numero Lotto;
    - **1.2** - Numero Vani;
    - **1.3** - Superficie;
    - **1.4** - Tipo Catasto;
    - **1.5** - Descrizione;
    
    
- **2.** Elimino le colonne ormai inutili;


- **3.** Rinomino le colonne del data frame;


- **4.** Conclusioni;


- **5.** Esporto il data frame in un file CSV;

<hr style="border:1px solid black">

**Importo le librerie che mi servono:**

In [1]:
import pandas as pd
import numpy as np
import re

#### Set pandas options:

In [2]:
pd.set_option('display.max_colwidth', None)
pd.options.display.max_rows = 5000
pd.options.display.max_columns = 1000
pd.options.display.float_format = '{:.2f}'.format

**Importo file CSV**

In [3]:
df = pd.read_csv("D:\\Master_Cefriel_DS_AI_Monolo\\0_Project_Work\\Dataset\\2_Dataset_Cleaning_1\\Dataset_Cleaning_1.csv")

### 1 - Correggo le colonne numeriche da dati ambigui o errati, ove possibile e sano le Null Values

- **1.1 - Colonna: NumLotto**

    - Dopo aver controllato tutti i valori presenti nella colonna "NumLotto", mantengo come valori corretti solo quelli che non presentano lettere, simboli e whitespaces;
    - Questo perchè il numero di un lotto può identificarsi solo come integer.

In [4]:
# Salvo la colonna NumLotto in una colonna di backup, sulla quale apporterò le successive modifiche:
df['NumLotto_Int'] = df['NumLotto']

# Sostituisco tutti i valori nella colonna che presentano simboli, con Null values:
df['NumLotto_Int'] = df['NumLotto_Int'].apply(lambda x: np.nan if pd.notna(x) and bool(re.search(r'[^\w\s]', x)) else x)

# Creo una Userd-Defined-Function che sostituisce con Null values ogni valore che presenta una lettera all'interno di ogni riga:
def replace_with_nan(row):
    if isinstance(row['NumLotto_Int'], str) and any(char.isalpha() for char in row['NumLotto_Int']):
        return np.nan
    return row['NumLotto_Int']

# Applico la UDF alla colonna:
df['NumLotto_Int'] = df.apply(replace_with_nan, axis=1)

# Sostiuisco tutti i valori che presentano _ con Null Values:
df['NumLotto_Int'] = df['NumLotto_Int'].replace(r'_', np.nan, regex=True)

# Sostiuisco tutti i valori che presentano whitespaces con Null Values:
df['NumLotto_Int'] = df['NumLotto_Int'].replace(r' ', np.nan, regex=True)

# Trasformo tutte le Null Values in 0:
df['NumLotto_Int'].fillna('0', inplace=True)

# Converto la colonna NumLotto_Int in formato Integer:
df['NumLotto_Int'] = df['NumLotto_Int'].astype('int64')

# Sostituisco di nuovo ora i valori uguali a 0 con Null Values:
df['NumLotto_Int'].replace(0, np.nan, inplace=True)
df['NumLotto_Int'] = df['NumLotto_Int'].astype('Int64', errors='ignore')

# Conto il numero delle Null Values finali ottenute:
print("Null Values totali: ",
      '\033[1m','\033[15;31;43m',
      df['NumLotto_Int'].isnull().sum(),
      '\033[0m')

#Rispetto ai valori interi ottenuti:
print("Numero Lotto in Integer format ottenuti: ",
      '\033[1m','\033[15;31;43m',
      df['NumLotto_Int'].notnull().sum(),
      '\033[0m')

Null Values totali:    4040 
Numero Lotto in Integer format ottenuti:    218696 


- **1.2 - Colonna: Numero Vani**

    - Dopo aver controllato i valori presenti nella colonna "NumeroVani", converto i caratteri utili a trasformare la colonna in un formato numerico, consono al tipo di dato che viene trattato in questo campo;
    - Infine, convertendo la colonna in decimale, converto gli 0 in valore assente, in quanto un bene non può non avere vani;
    - Nel caso non abbia vani, significa che il valore o manca, oppure non è soggetto a questo tipo di caratteristica, E.G. i terreni (LAND). 

In [5]:
# Salvo la colonna Numero Vani in una colonna di backup, sulla quale apporterò le successive modifiche:
df['NumeroVani_Float'] = df['NumeroVani']

# Sostituisco "," con "." per poi trasformare la colonna di backup in formato float:
df['NumeroVani_Float'] = df['NumeroVani_Float'].replace(',', '.', regex=True)
# Sostituisco "_" con "" per poi trasformare la colonna di backup in formato float:
df['NumeroVani_Float'] = df['NumeroVani_Float'].replace('-', '', regex=True)
# Converto la colonna da object a float32, 32 perchè non necessito di avere molti decimali nella medesima colonna:
df['NumeroVani_Float'] = df['NumeroVani_Float'].astype('float32')

# Sostituisco 0 con Null Values, in quanto un bene non può avere 0 vani, quindi lo leggo come valore assente:
df['NumeroVani_Float'].replace(0, np.nan, inplace=True)

# Conto il numero delle Null Values finali ottenute:
print("Null Values presenti: ", 
      '\033[1m',
      '\033[15;31;43m', 
      df['NumeroVani_Float'].isnull().sum(), 
      '\033[0m')

# Rispetto ai valori interi ottenuti:
print("Numero Vani presenti: ",
      '\033[1m','\033[15;31;43m',
      df['NumeroVani_Float'].notnull().sum(),
      '\033[0m')

Null Values presenti:    121164 
Numero Vani presenti:    101572 


- **1.3 - Colonna: Superficie**
    - Dopo aver controllato i valori presenti nella colonna "Superficie", noto che 89.246 valori sono uguali a 0 M2, su un totale di 222.736 beni;
    - Per bonificare questa colonna che appare scarsamente popolata, utilizzo i dati che provengono dalla colonna appena pulita "Numero Vani", in quanto è possibile risalire alla superficie di un bene moltiplicando il numero di vani * 20;
    - 20 quindi è il moltiplicatore utile a poter mappare la colonna superfice nel caso in cui questo dato non sia già disponibile;
    - 20 è il moltiplicatore indicato a livello aziendale, il quale prende spunto da studi derivanti dall'Agenzia Delle Entrate;
    - Inoltre, se il dato nella colonna superfice è una Null Values, si trasforma in 0.00 (tranne per i beni LAND, perchè non hanno vani), al fine di essere poi mappata con la moltiplicazione descritta sopra.

In [6]:
# Salvo la colonna Superficie in una colonna di backup, sulla quale apporterò le successive modifiche:
df['Superficie_Calcolata'] = df['Superficie']

# Sostituisco "," con "." per poi trasformare la colonna di backup in formato float:
df['Superficie_Calcolata'] = df['Superficie_Calcolata'].replace(',', '.', regex=True)

# Trasformo tutto in lower case:
df['Superficie_Calcolata'] = df['Superficie_Calcolata'].str.lower()
# Sostituisco le stringhe corrotte per poi trasformare la colonna di backup in formato float:
df['Superficie_Calcolata'] = df['Superficie_Calcolata'].str.replace('mq', '', regex=True)
df['Superficie_Calcolata'] = df['Superficie_Calcolata'].str.replace('92 \+ 49', '141', regex=True)
df['Superficie_Calcolata'] = df['Superficie_Calcolata'].str.replace('79\+21', '100', regex=True)
df['Superficie_Calcolata'] = df['Superficie_Calcolata'].str.replace('circa', '', regex=True)
df['Superficie_Calcolata'] = df['Superficie_Calcolata'].str.replace('106.99\+86.16', '193.15', regex=True)
# Converto la colonna da object a float32, 32 perchè non necessito di avere molti decimali nella medesima colonna:
df['Superficie_Calcolata'] = df['Superficie_Calcolata'].astype('float32')

# Trasformo le Null values in 0.00 nella colonna superfice se il campo Destinazione d'uso non è LAND:
def fill_nan_based_on_condition(df, condition_column, fill_column):
    df[fill_column] = np.where(df[condition_column] != "LAND", df[fill_column].fillna(0.00), df[fill_column])
    return df
# Applico la UDF creata sopra alla colonna superfice di backup:
df = fill_nan_based_on_condition(df, 'DestinazioneUso', 'Superficie_Calcolata')

# Mappo la colonna di backup laddove i valori sono uguali a 0.00, moltiplicando "Numero Vani" * 20:
def map_and_multiply(row):
    if row['Superficie_Calcolata'] == 0.00:
        return 20 * row['NumeroVani_Float']
    else:
        return row['Superficie_Calcolata']

# Applico la UDF alla colonna di backup:
df['Superficie_Calcolata'] = df.apply(map_and_multiply, axis=1)

# Conto il numero delle Null Values finali ottenute:
print("Null Values totali: ", 
      '\033[1m',
      '\033[15;31;43m', 
      df['Superficie_Calcolata'].isnull().sum(), 
      '\033[0m')

# Conto il numero delle superfici ancora uguali a 0:
print("Superfici uguali a 0 M2: ", 
      '\033[1m',
      '\033[15;31;43m', 
      (df['Superficie_Calcolata'] == 0.00).sum(), 
      '\033[0m')

# Rispetto ai valori interi ottenuti:
print("Superfici in Float format ottenute: ",
      '\033[1m','\033[15;31;43m',
      df['Superficie_Calcolata'].notnull().sum(),
      '\033[0m')

Null Values totali:    54326 
Superfici uguali a 0 M2:    0 
Superfici in Float format ottenute:    168410 


- **1.2.2 - Colonna: Numero Vani**
    - Adottando la stessa logica, posso ora applicarla ai valori attualmente assenti nel campo Numero Vani;
    - Dividendo la superfice / 20, per ottenere il numero dei vani, laddove in questo momento il valore è assente.
    - Il numero di vani può essere, o numero intero oppure aumentato di 0.5 per volta, quindi dopo la divisione è necessario approssimare il risultato ottenuto.

In [7]:
# Ottengo il numero di vani laddove siano assenti, dividendo la superfice del bene per 20:
df['NumeroVani_Float'] = df.apply(lambda row: row['Superficie_Calcolata'] / 20 if np.isnan(row['NumeroVani_Float'])
                                  else row['NumeroVani_Float'], axis=1)

# Creo una funzione per arrotondare per eccesso o per difetto i float ricavati dalla divisione precedente:
def numero_vani_round(value):
    if np.isnan(value):
        return value
    decimal_part = value - int(value)
    if decimal_part >= 0.75:
        return int(value) + 1.0
    elif decimal_part >= 0.25:
        return int(value) + 0.5
    else:
        return int(value)
    
# Applico la UDF alla collonna:
df['NumeroVani_Float'] = df['NumeroVani_Float'].apply(numero_vani_round)

# Conto il numero delle Null Values finali ottenute:
print("Null Values presenti: ", 
      '\033[1m',
      '\033[15;31;43m', 
      df['NumeroVani_Float'].isnull().sum(), 
      '\033[0m')

# Rispetto ai valori interi ottenuti:
print("Numero Vani presenti: ",
      '\033[1m','\033[15;31;43m',
      df['NumeroVani_Float'].notnull().sum(),
      '\033[0m')

# Ottengo il numero di valori inerenti al numero vani dei beni del data frame sanati rispetto a prima:
print("Numero Vani sanati rispetto a prima: ",
      '\033[1m','\033[15;31;43m',
      121164 - df['NumeroVani_Float'].isnull().sum(),
      '\033[0m')

Null Values presenti:    54311 
Numero Vani presenti:    168425 
Numero Vani sanati rispetto a prima:    66853 


- **1.4 - Colonna: Tipo Catasto**
    - Controllo la distribuzione dei valori presenti nella colonna Tipo Catasto;
    - La categoria T (Terreni) è ammessa solo per i beni inerenti alla destinazione d'uso LAND, in quanto facenti parte del catasto T (Terreni) e non F (Fabbricati);
    - Laddove questa associazione non fosse presente, mappare correttamente la colonna Tipo Catasto.
    - Trasformo il campo Tipo Catasto da "**T**" & "**F**" in un formato **Boolean**.

In [8]:
# Check iniziale per trovare incongruenze nei beni:
# Numero beni con catasto Terreni e destinazione d'uso Land:
df_t_l = df[(df['TipoCatasto'] == "T") & (df['DestinazioneUso'] == "LAND")]
print("Numero beni con catasto Terreni e destinazione d'uso Land:", '\033[1m','\033[15;31;43m', df_t_l.shape[0], '\033[0m')
# Numero beni con catasto fabbricati e destinazione d'uso Land:
df_f_l = df[(df['TipoCatasto'] == "F") & (df['DestinazioneUso'] == "LAND")]
print("Numero beni con catasto Fabbricati e destinazione d'uso Land:", '\033[1m','\033[15;31;43m', df_f_l.shape[0], '\033[0m')
# Numero beni con catasto terreni e con destinazione d'uso diverso da Land:
df_t_nl = df[(df['TipoCatasto'] == "T") & (df['DestinazioneUso'] != "LAND")]
print("Numero beni con catasto Terreni e destinazione d'uso non Land:", '\033[1m','\033[15;31;43m', df_t_nl.shape[0], '\033[0m')
print("Quindi ci sono 9766 beni coerenti. Mentre i beni non coerenti sono: ", '\033[1m','\033[15;31;43m',
      int(df_f_l.shape[0] + df_t_nl.shape[0]),'\033[0m', "i quali sono da eliminare dal data frame.", "\n")

# Filtro il mio data frame, al netto degli errori:
# Salvo l'indice del primo sub-data frame inerente ai beni fabbricati & land:
errore_1 = df_f_l.index
# Salvo l'indice del primo sub-data frame inerente ai beni Terreni & not-land:
errore_2 = df_t_nl.index
# Elimino dal mio data frame le righe contenenti i dati sbagliati, secondo i due indici settati prima:
df = df.drop(errore_1)
df = df.drop(errore_2)

#Ricontrollo il tutto:
# Numero beni con catasto Terreni e destinazione d'uso Land:
df_t_l_1 = df[(df['TipoCatasto'] == "T") & (df['DestinazioneUso'] == "LAND")]
print("Numero beni con catasto Terreni e destinazione d'uso Land:", '\033[1m','\033[15;31;43m', df_t_l_1.shape[0], '\033[0m')
# Numero beni con catasto fabbricati e destinazione d'uso Land:
df_f_l_1 = df[(df['TipoCatasto'] == "F") & (df['DestinazioneUso'] == "LAND")]
print("Numero beni con catasto Fabbricati e destinazione d'uso Land:", '\033[1m','\033[15;31;43m', df_f_l_1.shape[0], '\033[0m')
# Numero beni con catasto terreni e con destinazione d'uso diverso da Land:
df_t_nl_1 = df[(df['TipoCatasto'] == "T") & (df['DestinazioneUso'] != "LAND")]
print("Numero beni con catasto Terreni e destinazione d'uso non Land:",
      '\033[1m','\033[15;31;43m', df_t_nl_1.shape[0], '\033[0m')
print("Ci sono ancora i 9766 beni coerenti. Mentre i beni non coerenti sono ora stati eliminati in quanti pari a",
      '\033[1m','\033[15;31;43m', int(df_f_l_1.shape[0] + df_t_nl_1.shape[0]), '\033[0m')

Numero beni con catasto Terreni e destinazione d'uso Land:   9766 
Numero beni con catasto Fabbricati e destinazione d'uso Land:   761 
Numero beni con catasto Terreni e destinazione d'uso non Land:   155 
Quindi ci sono 9766 beni coerenti. Mentre i beni non coerenti sono:    916  i quali sono da eliminare dal data frame. 

Numero beni con catasto Terreni e destinazione d'uso Land:   9766 
Numero beni con catasto Fabbricati e destinazione d'uso Land:   0 
Numero beni con catasto Terreni e destinazione d'uso non Land:   0 
Ci sono ancora i 9766 beni coerenti. Mentre i beni non coerenti sono ora stati eliminati in quanti pari a   0 


In [9]:
# Infine Trasformo la colonna ormai pulita da errori Tipo Catasto in formato Boolean:
df['TipoCatasto_Bool'] = df['TipoCatasto'].apply(lambda x: 1 if x == 'F' else 0)
# Trasformo la colonna da integer a Boolean type:
df['TipoCatasto_Bool'] = df['TipoCatasto_Bool'].astype(bool)

# Controllo il risultato finale se tutto coincide:
t = df[df['TipoCatasto'] == "T"]
print("Numero beni con catasto Terreni:", '\033[1m','\033[15;31;43m', t.shape[0], '\033[0m')
f = df[df['TipoCatasto'] == "F"]
print("Numero beni con catasto Fabbricati:", '\033[1m','\033[15;31;43m', f.shape[0], '\033[0m', "\n")
# Controllo il risultato finale se tutto coincide:
Zero = df[df['TipoCatasto_Bool'] == False]
print("Numero beni con catasto boolean False:", '\033[1m','\033[15;31;43m', Zero.shape[0], '\033[0m')
Uno = df[df['TipoCatasto_Bool'] == True]
print("Numero beni con catasto boolean True:", '\033[1m','\033[15;31;43m', Uno.shape[0], '\033[0m')

Numero beni con catasto Terreni:   9766 
Numero beni con catasto Fabbricati:   212054  

Numero beni con catasto boolean False:   9766 
Numero beni con catasto boolean True:   212054 


- **1.5 - Colonna: Descrizione**
    - Ottengo la lunghezza della colonna Descrizione e salvo i valori in una nuova colonna;
    - Analizzo i valori ricavati;
    - Se ci sono valori estremamente troppo bassi, allora rimuovo le righe contenenti queste brevi descrizioni;

In [10]:
# Conto il numero di valori rimasti:
print("Il data frame ha N beni: ", '\033[1m','\033[15;31;43m', df.shape[0], '\033[0m')

# Ottengo la colonna nuova in format integer derivante dal numero di caratteri presenti nel campo descrizione:
df['Descrizione_Length'] = df['Descrizione'].apply(lambda x: len(x))

# Seleziono solo i valori minori di 4 caratteri:
df_corti = df[df['Descrizione_Length'] < 4]

# Conto il numero ricavato di beni con descrizioni corte:
print("Il numero totale di beni con descrizioni sotto i 4 caratteri è di: ", '\033[1m','\033[15;31;43m', df_corti.shape[0], '\033[0m')

# Trovo la relativa distribuzione
df_corti = df_corti['Descrizione'].value_counts().reset_index()
# Rinomino le colonne del data frame ottenuto:
df_corti = df_corti.rename(columns={"index" : "Stringa", "Descrizione" : "Count"})
# Esamino il risultato del data frame ottenuto stampandolo di seguito:
display(df_corti.head(10))

# Prima considerazione:
print("I valori da togliere sono quindi: ", '\033[1m','\033[15;31;43m', 8,
      '\033[0m', ", ovvero tutte le values che non sono inerenti alle descrizionid dei Box.")

# A questo punto, instanzio in una lista i valori da eliminare:
lista_drop = ["C/2", "A/3", "A/5"]
# Creo una mask per il successivo filtraggio dei valori:
drop_strings = df['Descrizione'].isin(lista_drop)
# Elimino i valori dal data frame principale:
df = df[~drop_strings]

# Conto il numero di valori rimasti:
print("Dopo il filtraggio il data frame ha N beni: ", '\033[1m','\033[15;31;43m', df.shape[0], '\033[0m')

Il data frame ha N beni:    221820 
Il numero totale di beni con descrizioni sotto i 4 caratteri è di:    99 


,Stringa,Count
0,Box,45
1,box,44
2,C/2,5
3,A/3,2
4,BOx,2
5,A/5,1


I valori da togliere sono quindi:    8  , ovvero tutte le values che non sono inerenti alle descrizionid dei Box.
Dopo il filtraggio il data frame ha N beni:    221812 


### 2 - Elimino le colonne che non servono più, filtrando il data frame con solo le colonne che mi sono utili:

In [11]:
# Seleziono solo le colonne che mi serve tenere, ovvero quelle descrittive e quelle numeriche pulite prima:
lista_colonne_tenere = ['DestinazioneUso',
                        'CategoriaCatastale',
                        'Tribunale',
                        'Provincia',
                        'Comune',
                        'NumLotto_Int',
                        'NumeroVani_Float',
                        'Superficie_Calcolata',
                        'TipoCatasto_Bool',
                        'Descrizione',
                        'Descrizione_Length']

# Filtro il data frame con le liste selezionate:
df_filtrato = df.loc[:, lista_colonne_tenere]

# Check colonne filtrate:
for col in df_filtrato.columns:
    print('\033[1m','\033[85;81;83m', col, '\033[0m')

  DestinazioneUso 
  CategoriaCatastale 
  Tribunale 
  Provincia 
  Comune 
  NumLotto_Int 
  NumeroVani_Float 
  Superficie_Calcolata 
  TipoCatasto_Bool 
  Descrizione 
  Descrizione_Length 


### 3 - Rinomino le colonne del data frame:

In [12]:
# Indico i nomi delle nuove colonne del data frame:
colonne_nomi = {'DestinazioneUso': 'Destinazione_Uso',
                    'CategoriaCatastale': 'Categoria_Catastale',
                    'Tribunale': 'Tribunale',
                    'Provincia': 'Provincia',
                    'Comune': 'Comune',
                    'NumLotto_Int': 'Numero_Lotto',
                    'NumeroVani_Float': 'Numero_Vani',
                    'Superficie_Calcolata': 'Superficie',
                    'TipoCatasto_Bool': 'Catasto_Fabbricati',
                    'Descrizione': 'Descrizione_Bene',
                    'Descrizione_Length' : 'N_Caratteri'}

# Applico i nuovi nomi al data frame:
df_filtrato.rename(columns = colonne_nomi, inplace=True)

# Check colonne rinominate:
for col in df_filtrato.columns:
    print('\033[1m','\033[85;81;83m', col, '\033[0m')

  Destinazione_Uso 
  Categoria_Catastale 
  Tribunale 
  Provincia 
  Comune 
  Numero_Lotto 
  Numero_Vani 
  Superficie 
  Catasto_Fabbricati 
  Descrizione_Bene 
  N_Caratteri 


### 4 - Conclusioni:

- Le **colonne numeriche** sono state trasformare tutte da object a Integer o Float Format:
    - **Numero_Lotto**: *Integer* ;
    - **Numero_Vani**: *Float* ;
    - **Superficie**: *Float* ;
    

- La colonne con due valori, sono state trasformate da object a boolean type:
    - **Catasto_Fabbricati**: *Boolean* ;
    - Da questa colonna sono stati **eliminati 916 dati** in quanto incongruenti;
    

- La colonne Descrizione invece è stata l'unica colonna Object rimasta Object. 
    - In essa sono state **eliminate le osservazioni troppo brevi**, droppando in totale **8 dati**.


- E' stata creata una nuova colonna, in base al numero di caratteri presenti nella colonna Descrizione.
    - **N_Caratteri**: *Integer* 
    

- Il data frame è ora composto da 11 colonne:
    - *6 colonne - Object* ;
    - *2 colonne - Integers* ;
    - *2 colonne - Floats* ;
    - *1 colonnna - Boolean* .
    
    
- Il data frame inizialmente comprendeva **222.736** Beni, ora dopo la fase di pulizia ne comprende **221.812**.
    - Sono stati rimossi **924** beni dal data frame.

### 5 - Export data frame file CSV:

In [13]:
df_filtrato_csv = df_filtrato.to_csv('D:\\Master_Cefriel_DS_AI_Monolo\\0_Project_Work\\Dataset\\3_Dataset_Cleaning_2\\Dataset_Cleaning_2.csv',
                                     index=False)